In [ ]:
%%writefile sumArrayGPU.cu
// CUDA kernel function
__global__ void my_cuda_kernel(int *input, int *output, int size) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < size) {
        output[tid] = input[tid] * 2;  // Example: Double each element
    }
}

// Wrapper function to call the CUDA kernel
extern "C" void my_cuda_function(int *input, int *output, int size) {
    // Allocate device memory
    int *d_input, *d_output;
    cudaMalloc((void**)&d_input, size * sizeof(int));
    cudaMalloc((void**)&d_output, size * sizeof(int));

    // Copy input data to device
    cudaMemcpy(d_input, input, size * sizeof(int), cudaMemcpyHostToDevice);

    // Launch CUDA kernel
    int threadsPerBlock = 256;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;
    my_cuda_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_input, d_output, size);

    // Copy result back to host
    cudaMemcpy(output, d_output, size * sizeof(int), cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_output);
}

Overwriting sumArrayGPU.cu


In [ ]:
!nvcc -arch=sm_75 -o sumArrayGPU.so -shared -Xcompiler -fPIC sumArrayGPU.cu

In [ ]:
import ctypes

# Load the CUDA library
cuda_lib = ctypes.CDLL('./sumArrayGPU.so')  # Update with the correct path

# Define the function prototype
cuda_lib.my_cuda_function.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_int), ctypes.c_int]
cuda_lib.my_cuda_function.restype = None

# Prepare data
input_data = [1, 2, 3, 4]
output_data = [0, 0, 0, 0]
size = len(input_data)

# Convert Python lists to ctypes arrays
input_array = (ctypes.c_int * size)(*input_data)
output_array = (ctypes.c_int * size)(*output_data)

# Call the CUDA function
cuda_lib.my_cuda_function(input_array, output_array, size)

# Print the result
result = list(output_array)
print("Result:", result)

Result: [2, 4, 6, 8]
